In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import minimize

In [ ]:
def LVmodel(y, t, alpha, beta, gamma, delta):
    u, v = y
    dydt = [(alpha-beta*v)*u, (-gamma+delta*u)*v]
    return dydt

def min_func(x, *args):
    t = args[0]
    u_e = args[1]
    v_e = args[2]
    
    alpha = x[0]
    beta = x[1]
    gamma = x[2]
    delta = x[3]

    y0 = [u_e[0], v_e[0]]
    y = odeint(LVmodel, y0, t, args=(alpha, beta, gamma, delta))
    delta = np.sum((y[:, 0]-u_e)**2)+np.sum((y[:, 1]-v_e)**2)
    return delta

def ODE(y, t, alpha, beta, gamma, delta):
    u, v, u_a, u_b, u_g, u_d, v_a, v_b, v_g, v_d = y
    dydt = [
        (alpha-beta*v)*u,
        (-gamma+delta*u)*v,
        
        u+(alpha-beta*v)*u_a+(-beta*u)*v_a,
        -v*u+(alpha-beta*v)*u_b+(-beta*u)*v_b,
        (alpha-beta*v)*u_g+(-beta*u)*v_g,
        (alpha-beta*v)*u_d+(-beta*u)*v_d,
        
        delta*v*u_a+(-gamma+delta*u)*v_a,
        delta*v*u_b+(-gamma+delta*u)*v_b,
        -v+delta*v*u_g+(-gamma+delta*u)*v_g,
        u*v+delta*v*u_d+(-gamma+delta*u)*v_d,
        ]
    return dydt

def min_func_jac(x, *args):
    t = args[0]
    u_e = args[1]
    v_e = args[2]
    
    alpha = x[0]
    beta = x[1]
    gamma = x[2]
    delta = x[3]
    
    theta0 = [u_e[0], v_e[0], 0, 0, 0, 0, 0, 0, 0, 0]

    zzz = odeint(ODE, y0=theta0, t=t, args=(alpha, beta, gamma, delta))
    res = np.zeros((4))
    for i in range(0, 4):
        res[i] += np.sum((zzz[:,0]-u_e)*zzz[:,2+i])
        res[i] += np.sum((zzz[:,1]-v_e)*zzz[:,2+4+i])
        res[i] *= 2
    return res

In [ ]:
ts = 0
te = 20
n = 200
t = np.linspace(ts, te, n)

alpha = 0.42
beta = 0.14
gamma = 0.79
delta = 0.17


y0 = [15, 10]

sol = odeint(LVmodel, y0, t, args=(alpha, beta, gamma, delta))
u_e = sol[:, 0]
v_e = sol[:, 1]
plt.plot(t, sol[:, 0], 'b')
plt.plot(t, sol[:, 1], 'g')

In [ ]:
plt.plot(sol[:, 0], sol[:, 1])
prcnt = 0.05
u_e += prcnt*sol[:, 0]*np.random.randn(n)
v_e += prcnt*sol[:, 1]*np.random.randn(n)

In [ ]:
for i in range(0, 10):
    rand_x0 = np.random.rand(4)
    print('x0 = ', rand_x0)
    res = minimize(fun=min_func, x0=rand_x0, args=(t, u_e, v_e), method='Nelder-Mead', tol=1e-3)
    print('x = ', res.x)
    print('err = ', res.fun)
    print('nit = ', res.nit)
    print()

In [ ]:
for i in range(0, 10):
    rand_x0 = np.random.rand(4)
    print('x0 = ', rand_x0)
    res = minimize(fun=min_func, x0=rand_x0, args=(t, u_e, v_e), method='TNC', jac=min_func_jac)
    print('x = ', res.x)
    print('err = ', res.fun)
    print('nit = ', res.nit)
    print()

In [ ]:
xn = [0.41572561, 0.13543324, 0.76847345, 0.16055372]
tn = np.linspace(0, 80, 400)
sol_new = odeint(LVmodel, y0, t, args=(xn[0], xn[1], xn[2], xn[3]))


plt.plot(sol[:, 0], sol[:, 1])
plt.plot(sol_new[:, 0], sol_new[:, 1])